In [1]:
%matplotlib inline
import os
import random
import matplotlib.pyplot as plt
import geopandas as gpd
import pandas as pd
import gtfs_kit as gk
import numpy as np
import osmnx as ox
import networkx as nx
from tqdm import tqdm
from shapely.geometry import Polygon, LineString, Point
import warnings
warnings.filterwarnings('ignore')
from pyspark import SparkContext,SparkConf
from pyspark.sql import SQLContext
from pyspark.sql import Row, SparkSession
from pyspark.sql.types import IntegerType
from pyspark.sql import functions as F
from pyspark import SparkConf
import pandas as pd
import pickle
from tqdm import tqdm
spark = SparkSession.builder.config('spark.executor.cores', '8').config('spark.executor.memory', '80g')\
        .config("spark.sql.session.timeZone", "UTC").config('spark.driver.memory', '40g').master("local[26]")\
        .appName("wego-daily").config('spark.driver.extraJavaOptions', '-Duser.timezone=UTC').config('spark.executor.extraJavaOptions', '-Duser.timezone=UTC')\
        .config("spark.sql.datetime.java8API.enabled", "true").config("spark.sql.execution.arrow.pyspark.enabled", "true")\
        .config("spark.sql.autoBroadcastJoinThreshold", -1)\
        .config("spark.driver.maxResultSize", 0)\
        .config("spark.shuffle.spill", "true")\
        .getOrCreate()
spark.sparkContext.setLogLevel("ERROR")

/home/jptalusan/anaconda3/envs/py39/lib/python3.9/site-packages/geopandas/_compat.py:123: UserWarning: The Shapely GEOS version (3.10.3-CAPI-1.16.1) is incompatible with the GEOS version PyGEOS was compiled with (3.11.1-CAPI-1.17.1). Conversions between both will be slow.
  warnings.warn(


23/01/06 15:35:17 WARN Utils: Your hostname, scope-vanderbilt resolves to a loopback address: 127.0.1.1; using 10.2.218.69 instead (on interface enp8s0)
23/01/06 15:35:17 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/01/06 15:35:17 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


# Files for `EmpiricalTravelModelLookup.py`
```
disruption_path = f'{base_dir}/common/disruption_probabilities.pkl'
self.sampled_disruption = pd.read_pickle(disruption_path)

self.logger = logger

with open(f'{base_dir}/common/sampled_travel_times_dict.pkl', 'rb') as handle:
    self.sampled_travel_time = pickle.load(handle)

with open(f'{base_dir}/common/stops_tt_dd_node_dict.pkl', 'rb') as handle:
    self.stops_tt_dd_dict = pickle.load(handle)

with open(f'{base_dir}/common/stops_node_matching_dict.pkl', 'rb') as handle:
    self.stop_nodes_dict = pickle.load(handle)
```

## Disruption dataset

In [ ]:
# Get service disruption dataset
fp = os.path.join('data/Service Disruptions_07_2019_08_2022.csv')
disruptions_df = pd.read_csv(fp)
disruptions_df.head()
disruptions_df['DATETIME'] = disruptions_df['DATE'] + ' ' + disruptions_df['TIME']
disruptions_df['DATE'] = pd.to_datetime(disruptions_df['DATE'], format='%m/%d/%y', errors='coerce')
disruptions_df['TIME'] = pd.to_datetime(disruptions_df['TIME'], format='%H:%M:%S', errors='coerce')
disruptions_df['DATETIME'] = pd.to_datetime(disruptions_df['DATETIME'], format='%m/%d/%y %H:%M:%S', errors='coerce')

# Remove mechanical and weather disruptions
disruptions_df = disruptions_df.query("REASON != 'Mechanical' and REASON != 'Weather'")

disruptions_df['BLOCK'] = disruptions_df['BLOCK'].astype('int32')# disruptions_sp = disruptions_sp.withColumn("BLOCK", F.col("BLOCK").cast(IntegerType()))
disruptions_counts_df = disruptions_df.groupby('START_STOP_ABBR').agg('count')[['REASON']].reset_index().sort_values('REASON')

# Count the number of trips throughout this time
start_date = disruptions_df.sort_values(by=['DATETIME']).iloc[0]['DATETIME']
end_date   = disruptions_df.sort_values(by=['DATETIME']).iloc[-1]['DATETIME']

print(start_date, end_date)

# Filter APC data on these dates
# # filter subset
get_columns = ['transit_date', 'trip_id', 'departure_time', 'stop_id_original']
get_str = ", ".join([c for c in get_columns])

f = os.path.join('/home/jptalusan/mta_stationing_problem/data/processed/apc_weather_gtfs.parquet')
apcdata = spark.read.load(f)
apcdata.createOrReplaceTempView("apc")
query = f"""
SELECT {get_str}
FROM apc
WHERE (transit_date >= '{start_date.date()}') AND (transit_date <= '{end_date.date()}')
"""
apcdataafternegdelete = spark.sql(query)
apcdataafternegdelete = apcdataafternegdelete.dropna()
trips_df = apcdataafternegdelete.toPandas()
trips_df = trips_df.groupby('stop_id_original').agg('count').sort_values('trip_id').reset_index()

# Merging them toegether
merged_df = pd.merge(trips_df, disruptions_counts_df[['START_STOP_ABBR', 'REASON']], left_on='stop_id_original', right_on='START_STOP_ABBR')
merged_df['probability'] = merged_df['REASON'] / merged_df['transit_date']
merged_df['probability'] = merged_df['probability']/merged_df['probability'].max()
merged_df.sort_values('probability').tail(10)

all_stop_probabilities = trips_df[['stop_id_original']]
all_stop_probabilities = pd.merge(all_stop_probabilities, merged_df[['stop_id_original', 'probability']], on='stop_id_original', how='outer').fillna(0)
all_stop_probabilities.sort_values('probability')

# fp = os.path.join('results/disruption_probabilities.pkl')
# all_stop_probabilities.to_pickle(fp)
all_stop_probabilities


## Travel times

In [ ]:
# from multiprocessing import Pool, cpu_count
# import datetime as dt
# import pandas as pd
# import random

# def get_traveltimes(tdf):
#     tdf = tdf.sort_values('stop_sequence')
#     # HACK: For null arrival times in the middle.
#     tdf['stop_sequence'] = range(1, len(tdf) + 1)
#     if len(tdf) <= 2:
#         return pd.DataFrame()
#     # HACK: This is for correcting the issue that the first stop's arrival_time starts much earlier than the scheduled time

#     tdf = tdf.reset_index(drop=True)
#     # tdf['scheduled_timestamp'] = (tdf['arrival_time'] - dt.datetime(1970,1,1)).dt.total_seconds()
#     tdf['arrival_at_next_stop'] = tdf.arrival_time.shift(-1)
#     tdf['time_to_next_stop'] = tdf['arrival_at_next_stop'] - tdf['departure_time']
#     tdf.at[0, 'time_to_next_stop'] = (tdf.at[1, 'arrival_time'] - tdf.at[0, 'scheduled_time']).total_seconds()
#     tdf = tdf.drop('scheduled_timestamp', axis=1)
#     tdf = tdf.fillna(0)
#     return tdf
    
# def applyParallel(dfGrouped, func):
#     with Pool(cpu_count()-2) as p:
#         ret_list = p.map(func, [group for name, group in dfGrouped])
#     return pd.concat(ret_list)

# start_date = dt.datetime.strptime('2020-03-21 06:28:00', '%Y-%m-%d %H:%M:%S')
# end_date = dt.datetime.strptime('2022-03-22 06:38:00', '%Y-%m-%d %H:%M:%S')

# # Read APC data.
# f = os.path.join('/home/jptalusan/mta_stationing_problem/data/processed/apc_weather_gtfs_20221216.parquet')
# apcdata = spark.read.load(f)
# get_columns = ['trip_id', 'transit_date', 'arrival_time', 'departure_time', 'block_abbr', 'scheduled_time', 'vehicle_id',
#               'stop_sequence', 'stop_id_original', 'route_id', 'route_direction_name']
# get_str = ", ".join([c for c in get_columns])
# apcdata.createOrReplaceTempView("apc")
# # # filter subset
# query = f"""
#        SELECT {get_str}
#        FROM apc
#        WHERE (transit_date >= '{start_date.date()}') AND (transit_date <= '{end_date.date()}')
#        """
# print(query)
# apcdata = spark.sql(query)
# apcdata = apcdata.withColumn("route_id_direction", F.concat_ws('_',apcdata.route_id, apcdata.route_direction_name))
# apcdata = apcdata.drop('route_id', 'route_direction_name')
# baseline_data = apcdata.toPandas()
# baseline_data = baseline_data.dropna(subset=['arrival_time', 'departure_time'])

# # Compute travel times
# out_arr = applyParallel(baseline_data.groupby(['block_abbr', 'route_id_direction', 'transit_date', 'trip_id']), get_traveltimes)
# tdf = out_arr.groupby(['route_id_direction', 'block_abbr', 'stop_sequence', 'stop_id_original']).agg({'time_to_next_stop':list})
# # fp = os.path.join('results/travel_time_by_scheduled_time.pkl')
# # tdf.to_pickle(fp)

# random.seed(100)

# tdf = tdf.reset_index()
# tdf['sampled_travel_time'] = tdf.reset_index()['time_to_next_stop'].apply(lambda x: random.choice(x))
# tdf['sampled_travel_time'] = abs(tdf['sampled_travel_time'])
# # fp = os.path.join('results/sampled_travel_times.pkl')
# # tdf.to_pickle(fp)

## stops_tt_dd_node_dict
* Run `pair_tt_dd.py`: this requires a multiprocessing which is why its .py script

In [ ]:
# import pandas as pd
# import pickle

# f = os.path.join('/home/jptalusan/mta_stationing_problem/data/processed/apc_weather_gtfs_20221216.parquet')
# apcdata = spark.read.load(f)
# apcdata.columns
# get_columns = ['stop_sequence', 'stop_id_original', 'stop_name', 'map_latitude', 'map_longitude']
# get_str = ", ".join([c for c in get_columns])
# apcdata.createOrReplaceTempView("apc")

# # # filter subset
# query = f"""
# SELECT {get_str}
# FROM apc
# LIMIT 1000
# """
# apcdata = spark.sql(query)
# apcdata = apcdata.drop_duplicates(['stop_id_original'])
# apcdf = apcdata.toPandas()

# fp = os.path.join('data', 'shapefiles', "tncounty")
# gdf_county = gpd.read_file(fp)
# gdf_dav = gdf_county[gdf_county["NAME"] == "Davidson"]
# gdf_dav = gdf_dav.to_crs("EPSG:4326")

# G = ox.graph_from_polygon(gdf_dav.geometry.iloc[0], network_type='drive')
# G = ox.add_edge_speeds(G)
# G = ox.add_edge_travel_times(G)

# apcdf['nearest_node'] = ox.nearest_nodes(G, apcdf['map_longitude'], apcdf['map_latitude'])
# apcdf['nearest_edge'] = ox.nearest_edges(G, apcdf['map_longitude'], apcdf['map_latitude'])

# fp = os.path.join('results', 'stops_node_matching.pkl')
# apcdf.to_pickle(fp)
# # fp = os.path.join('data', 'davidson_graph.graphml')
# # ox.save_graphml(G, fp)

> Run `pair_tt_dd.py` after cell above...

## stops_node_matching_dict.pkl

In [ ]:
# fp = '/home/jptalusan/mta_simulator/code/data/stops_node_matching.pkl'
# stop_nodes = pd.read_pickle(fp)
# stop_nodes = stop_nodes.set_index('stop_id_original')
# stop_nodes_dict = stop_nodes.drop(['stop_sequence', 
#                                    'stop_name', 'map_latitude', 
#                                    'map_longitude', 
#                                    'nearest_edge'], axis=1).to_dict('index')
# with open('results/stops_node_matching_dict.pkl', 'wb') as handle:
#     pickle.dump(stop_nodes_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)

# Create trip and vehicle schedules and passenger chains

Run `generate_day_trips.py`
This will generate files in:
* `data_generation/results/test_data`

# Create config and execute files

In [ ]:
#### SAMPLE CONFIG.json ####

# {
#   "starting_date_str": "20210607",
#   "iter_limit": 200,
#   "pool_thread_count": 10,
#   "mcts_discount_factor": 0.99997,
#   "uct_tradeoff": 8074255,
#   "lookahead_horizon_delta_t": 3600,
#   "rollout_horizon_delta_t": 3600,
#   "allowed_computation_time": 15,
#   "vehicle_count": "",
#   "oracle": false,
#   "method": "MCTS",
#   "use_intervals": true,
#   "use_timepoints": true,
#   "save_metrics": false,
#   "send_mail": true,
#   "reallocation": true,
#   "early_end": false,
#   "scenario": "1B",
#   "mcts_log_name": "20210607_ALL_IT200_nobreak"
# }

#### SAMPLE EXECUTOR.SH ####

# session="20210607"

# # Check if the session exists, discarding output
# # We can check $? for the exit status (zero for success, non-zero for failure)
# tmux has-session -t $session 2>/dev/null

# if [ $? != 0 ]; then
#   # Set up your session
#   tmux new-session -d -s $session
# else
#   tmux kill-session -t $session
# fi

# # ################################### WINDOW 0 ###################################

# window=0
# tmux rename-window -t $session:$window '1'
# tmux send-keys 'conda activate py39' 'C-m'
# cfile="configs/all_served"
# logName=${cfile#*/}
# tmux send-keys 'python run_mcts_no_inject.py -c '$cfile'' C-m
# tmux send-keys 'python emailer.py -c '$cfile'' C-m
# tmux send-keys 'tmux capture-pane -pJ -S - > 'logs/$logName'.log' C-m

# tmux split-window -h

# tmux send-keys 'conda activate py39' 'C-m'
# cfile="configs/all_served_100"
# logName=${cfile#*/}
# tmux send-keys 'python run_mcts_no_inject.py -c '$cfile'' C-m
# tmux send-keys 'python emailer.py -c '$cfile'' C-m
# tmux send-keys 'tmux capture-pane -pJ -S - > 'logs/$logName'.log' C-m
# tmux a -t $session


In [1]:
# Create json configs
import pandas as pd
import json
import numpy as np

def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]


date_range = pd.date_range('2022-10-09', '2022-10-09', freq='24h')
# date_range = pd.date_range('2021-12-15', '2021-12-15', freq='24h')
date_range = [dr.strftime('%Y-%m-%d') for dr in date_range]

configs_dir = 'results/test_data/configs/'
config_names = []
for date in date_range:
    date = date.replace("-", "")
    for iters in [100]:
        for mean_label, mean in [('P025', 118274608), ('P05', 59137304), ('P1', 29568652), ('P2', 14784326), ('P4', 7392163)]:
            mcts_log_name = f"{date}_{iters}IT_{mean_label}M_10CAP"
            
            config = dict()
            config["starting_date_str"] = date
            config["iter_limit"] = iters
            config["pool_thread_count"] = 10
            config["uct_tradeoff"] = mean
            config["mcts_discount_factor"] = 0.99997
            config["lookahead_horizon_delta_t"] = 3600
            config["rollout_horizon_delta_t"] = 3600
            config["allowed_computation_time"] = 15
            config["vehicle_count"] = "10CAP"
            config["oracle"] = False
            config["method"] = "MCTS"
            config["use_intervals"] = True
            config["use_timepoints"] = True
            config["send_mail"] = True
            config["save_metrics"] = False
            config["reallocation"] = True
            config["scenario"] = "1B"
            config["mcts_log_name"] = mcts_log_name
            
            with open(f'{configs_dir}/mcts_{date}_{iters}IT_{mean_label}M.json', 'w') as fp:
                json.dump(config, fp, indent=2)
                
            config_names.append(mcts_log_name)

devices = 1
windows = 2

date_range = list(np.array_split(date_range, devices))
for device in range(devices):
    lines = []
    d = list(np.array_split(date_range[device], windows))
    
    lines.append(f"session=device_{device}_200IT")
    lines.append("tmux has-session -t $session 2>/dev/null")
    lines.append("if [ $? != 0 ]; then")
    lines.append("  tmux new-session -d -s $session")
    lines.append("else")
    lines.append("  tmux kill-session -t $session")
    lines.append("fi")
    lines.append("")
    
    lines.append(f"window=0")
    # lines.append("tmux rename-window -t $session:$window '1'")
    
    for window in range(windows):
        lines.append("tmux send-keys 'conda activate py39' 'C-m'")
        lines.append("")
        
        for iter in [100]:
            for _d in d[window]:
                for mean_label, mean in [('P025', 118274608), ('P05', 59137304), ('P1', 29568652), ('P2', 14784326), ('P4', 7392163)]:
                    mcts_log_name = f"mcts_{_d.replace('-', '')}_{iter}IT_{mean_label}M"
                    lines.append(f"cfile=\"configs/{mcts_log_name}\"")
                    lines.append("logName=${cfile#*/}")
                    lines.append("tmux send-keys 'python run_mcts_no_inject.py -c '$cfile'' C-m")
                    lines.append("tmux send-keys 'python emailer.py -c '$cfile'' C-m")
                    lines.append("tmux send-keys 'tmux capture-pane -pJ -S - > 'logs/$logName'.log' C-m")
                    lines.append("")
        
        if window < windows - 1:
            lines.append("tmux split-window -h")
        lines.append("")

    lines.append("tmux select-layout tiled")
    lines.append("tmux a -t $session")
    
    with open(f'results/test_data/executers/execute_{device}_100IT.sh', 'w') as f:
        for line in lines:
            f.write(f"{line}\n")

### Baselines

In [6]:
# Create json configs
import pandas as pd
import json
import numpy as np

def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]


date_range = pd.date_range('2022-10-01', '2022-10-31', freq='24h')
# date_range = pd.date_range('2021-12-15', '2021-12-15', freq='24h')
date_range = [dr.strftime('%Y-%m-%d') for dr in date_range]

configs_dir = 'results/test_data/configs/'
config_names = []
for date in date_range:
    date = date.replace("-", "")
    for iters in [0]:
        mcts_log_name = f"BL_{date}_{iters}IT_10CAP_NOREALLOC"
        config = dict()
        config["starting_date_str"] = date
        config["iter_limit"] = iters
        config["pool_thread_count"] = 10
        config["uct_tradeoff"] = 31344050
        config["mcts_discount_factor"] = 0.99997
        config["lookahead_horizon_delta_t"] = 3600
        config["rollout_horizon_delta_t"] = 3600
        config["allowed_computation_time"] = 15
        config["vehicle_count"] = "10CAP"
        config["oracle"] = True
        config["method"] = "baseline"
        config["use_intervals"] = True
        config["use_timepoints"] = True
        config["send_mail"] = True
        config["save_metrics"] = False
        config["reallocation"] = False
        config["scenario"] = "1B"
        config["mcts_log_name"] = mcts_log_name
        
        with open(f'{configs_dir}/bl_{date}_{iters}IT.json', 'w') as fp:
            json.dump(config, fp, indent=2)
            
        config_names.append(mcts_log_name)

devices = 1
windows = 4

date_range = list(np.array_split(date_range, devices))
for device in range(devices):
    lines = []
    d = list(np.array_split(date_range[device], windows))
    
    lines.append(f"session=device_{device}_200IT")
    lines.append("tmux has-session -t $session 2>/dev/null")
    lines.append("if [ $? != 0 ]; then")
    lines.append("  tmux new-session -d -s $session")
    lines.append("else")
    lines.append("  tmux kill-session -t $session")
    lines.append("fi")
    lines.append("")
    
    lines.append(f"window=0")
    # lines.append("tmux rename-window -t $session:$window '1'")
    
    for window in range(windows):
        lines.append("tmux send-keys 'conda activate py39' 'C-m'")
        lines.append("")
        
        for iter in [200]:
            for _d in d[window]:
                mcts_log_name = f"bl_{_d.replace('-', '')}_{iter}IT"
                lines.append(f"cfile=\"configs/{mcts_log_name}\"")
                lines.append("logName=${cfile#*/}")
                lines.append("tmux send-keys 'python run_mcts_no_inject.py -c '$cfile'' C-m")
                # lines.append("tmux send-keys 'python emailer.py -c '$cfile'' C-m")
                lines.append("tmux send-keys 'tmux capture-pane -pJ -S - > 'logs/$logName'.log' C-m")
                lines.append("")
        
        if window < windows - 1:
            lines.append("tmux split-window -h")
        lines.append("")

    lines.append("tmux select-layout tiled")
    lines.append("tmux a -t $session")
    
    with open(f'results/test_data/executers/execute_bl_{device}_0IT.sh', 'w') as f:
        for line in lines:
            f.write(f"{line}\n")